In [6]:
import os
import tensorflow as tf
from tensorflow import keras

In [7]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

# 二维矩阵转一维矩阵，并且做归一化
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [8]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                  # todo SparseCategoricalCrossentropy 和 binary....
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    return model


In [9]:
model = create_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


在训练期间保存模型。checkpoint

In [10]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个保存模型权重的回调
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images, train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

2021-08-12 16:55:03.988794: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-12 16:55:03.993456: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-12 16:55:04.126114: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
32/32 [==============================] - 1s 17ms/step - loss: 1.1588 - accuracy: 0.6790 - val_loss: 0.7494 - val_accuracy: 0.7680

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
 1/32 [..............................] - ETA: 0s - loss: 0.5653 - accuracy: 0.8125

2021-08-12 16:55:04.906048: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 11ms/step - loss: 0.4235 - accuracy: 0.8850 - val_loss: 0.5144 - val_accuracy: 0.8450

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 11ms/step - loss: 0.2761 - accuracy: 0.9250 - val_loss: 0.4619 - val_accuracy: 0.8560

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 11ms/step - loss: 0.2075 - accuracy: 0.9490 - val_loss: 0.4405 - val_accuracy: 0.8640

In [14]:
# 创建一个未训练的新模型
model = create_model()

# 评估模型
model.evaluate(test_images, test_labels)

32/32 [==============================] - 0s 5ms/step - loss: 2.3987 - accuracy: 0.0630


2021-08-12 17:00:51.241259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


[2.3986501693725586, 0.06300000101327896]

In [15]:
# 加载由checkpoint保存的权重
model.load_weights(checkpoint_path)

# 重新评估
model.evaluate(test_images, test_labels)

32/32 [==============================] - 0s 6ms/step - loss: 0.3941 - accuracy: 0.8720


[0.39410772919654846, 0.8720000386238098]

In [16]:
# checkpoint 参数设置

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 每5个epochs保存一次
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_weights_only=True,
                                                 period=5)

model = create_model()

# todo 使用 `checkpoint_path` 格式保存权重
model.save_weights(checkpoint_path.format(epoch=0))

# 使用新的回调训练模型
model.fit(train_images,
          train_labels,
          epochs=50,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

2021-08-12 17:23:21.640449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-08-12 17:23:21.951236: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [19]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [21]:
# 再创建一个新模型，并加载最新的checkpoint文件
model = create_model()
model.load_weights(latest)
model.evaluate(test_images, test_labels)

27/32 [========================>.....] - ETA: 0s - loss: 0.4942 - accuracy: 0.8796

2021-08-12 17:30:30.498320: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 6ms/step - loss: 0.5038 - accuracy: 0.8720


[0.5038260221481323, 0.8720000386238098]

#### 手动保存权重

In [ ]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建模型实例
model = create_model()

# 恢复权重
model.load_weights('./checkpoints/my_checkpoint')

# 评估模型
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

#### 保存整个模型：SavedModel 格式

In [22]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
 9/32 [=======>......................] - ETA: 0s - loss: 1.9045 - accuracy: 0.3542

2021-08-12 17:45:03.601043: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 7ms/step - loss: 1.1713 - accuracy: 0.6420
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4297 - accuracy: 0.8740
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2904 - accuracy: 0.9210

In [23]:
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [26]:
# 从保存的模型重新加载
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

new_model.evaluate(test_images, test_labels, verbose=1)

print(new_model.predict(test_images).shape)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 [==============================] - 0s 5ms/step - loss: 0.4456 - accuracy: 0.8600

2021-08-12 18:13:41.565267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


(1000, 10)


2021-08-12 18:13:41.790728: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


#### 保存整个模型：HDF5 格式

In [27]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 保存为 HDF5 文件
model.save('my_model.h5')

Epoch 1/5
11/32 [=========>....................] - ETA: 0s - loss: 1.8557 - accuracy: 0.4233

2021-08-12 18:22:29.324693: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 8ms/step - loss: 1.1808 - accuracy: 0.6520
Epoch 2/5
32/32 [==============================] - 0s 7ms/step - loss: 0.4172 - accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2939 - accuracy: 0.9180

In [29]:
new_model = tf.keras.models.load_model('my_model.h5')

new_model.summary()

new_model.evaluate(test_images, test_labels)


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
27/32 [========================>.....] - ETA: 0s - loss: 0.4243 - accuracy: 0.8565

2021-08-12 18:33:43.699663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


32/32 [==============================] - 0s 6ms/step - loss: 0.4318 - accuracy: 0.8530


[0.4317818284034729, 0.8530000448226929]